In [2]:
import pandas as pd 
import numpy as np


In [3]:
data = pd.read_csv('hadoop.csv')


In [4]:
data = data.drop(['idx','access_date','access_time','access_page','access_referer','item_name'],axis=1)

In [5]:
unique_ip = data[data['item_type']=='cart']['access_ip'].unique()

## Make Dict_ip insert key is ip item is values

In [6]:
dict_ip = dict.fromkeys(unique_ip)   #dict key 생성

for i in dict_ip.keys():
    dict_ip[i] = []

#-----------------------------------------------#

for i in range(len(data)):          #dict value 삽입
    if (data['access_ip'].iloc[i] in dict_ip.keys()):
        key = data['access_ip'].iloc[i]
        dict_ip[key].append(data['item_id'].iloc[i])

In [7]:
for i,j in dict_ip.items():
    dict_ip[i] = list(map(str,j))        #dict value를 string으로 매핑
    
#------------------------------------------------#
#dict의 벨류가 4 미만이면 삭제
dict_ip = {key:value for key, value in dict_ip.items() if len(value) >4} 

In [8]:
cart_item = []       #dict의 마지막 아이템을 카트 아이템배열에 넣음
for i,j in dict_ip.items():
    cart_item.append(j[-1])

In [9]:
columns_name = []      #column이름을 string으로 변환
for i in range(14):
    columns_name.append(str(i))

## Make DataFrame New

In [10]:
df = pd.DataFrame.from_dict(dict_ip,orient='index',columns=columns_name)
df['13'] = cart_item

In [11]:
mask = df.applymap(lambda x: x is None)         #None값들 삭제
cols = df.columns[(mask).any()]
for col in df[cols]:
    df.loc[mask[col], col] = ''


In [12]:
item_list = []                 #item들을 뽑아서 한 row씩 한 리스트로 묶어 배열 생성
for i in range(len(df)):
    item_list.append(df.iloc[i,:].tolist())

# Word2Vec

In [13]:
from gensim.models import Word2Vec
from gensim.test.utils import common_texts, get_tmpfile

In [14]:
path = get_tmpfile('word2vec.model')

In [15]:
model = Word2Vec(item_list, size=1, window=5, min_count=1,workers=4)
model.save('word2vec.model')

In [16]:
model = Word2Vec.load('word2vec.model')

In [17]:
model.train([['0','12']],total_examples=1 , epochs=1)

(0, 2)

In [18]:
word_vectors = model.wv  
vocabs = word_vectors.vocab.keys() #vocabs has keys that is item number

#word_vectors_list has values that is vector value from item number
word_vectors_list = [word_vectors[v] for v in vocabs] 


In [19]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

In [20]:
# def plot_2d_graph(vocabs, xs, ys):
#     plt.figure(figsize=(12,6))
#     plt.scatter(xs,ys,marker='o')
#     for i, v in enumerate(vocabs):
#         plt.annotate(v,xy=(xs[i],ys[i]))

In [21]:
#pca = PCA(n_components=2)

In [22]:
# xys = pca.fit_transform(word_vectors_list)
# xs = xys[0]
# ys = xys[1]

In [23]:
# plot_2d_graph(vocabs,xs,ys)

In [24]:
for i in range(len(df)):            #word2vec value를 df로 넣음
    for j in range(len(df.iloc[i])):
        key=str(df.iloc[i,j])
        df.iloc[i,j] = word_vectors[key].tolist()[0]

In [25]:
df.index = range(len(df))

In [26]:
df = df.apply(pd.to_numeric)

# Data split && Learning

In [27]:
from sklearn.model_selection import train_test_split
import xgboost as xgb

In [28]:
X_data = df.drop('13',axis=1)
y_data = df['13']

In [29]:
seed = 7
test_size = 0.3
X_train,X_test,y_train,y_test = train_test_split(X_data,y_data,test_size=test_size, random_state=seed)

In [36]:
model1 = xgb.XGBClassifier()
model2= xgb.XGBClassifier(num_class = 828, n_estimators=100,max_depth=8,learning_rate=0.1,subsample=0.5)
eval_set = [(X_train, y_train), (X_test, y_test)]

In [37]:
model2.fit(X_train,y_train, eval_metric=["error", "logloss"], eval_set=eval_set, verbose=False)

ValueError: y contains previously unseen labels: [0.38278645277023315, 0.4481535255908966, 0.5450780391693115, 0.5601896047592163, 0.6879556179046631, 0.6989572644233704, 0.7717557549476624, 0.8054739236831665, 0.8320490717887878, 0.8357860445976257, 0.8387987613677979, 0.8609676361083984, 0.8807662129402161, 0.8855785131454468, 0.8855860829353333, 0.8878364562988281, 0.8913403749465942, 0.8951379656791687, 0.935180127620697, 0.9387589693069458, 0.9408422708511353, 0.9721554517745972, 0.9793010354042053, 0.9812204241752625, 0.9902193546295166, 0.9916657209396362, 0.997627854347229, 0.9989503026008606, 1.0047544240951538, 1.0419447422027588, 1.0487127304077148, 1.0493791103363037, 1.0509411096572876, 1.0588736534118652, 1.078368067741394, 1.08403480052948, 1.0856430530548096, 1.0885889530181885, 1.0930970907211304, 1.0987509489059448, 1.126847505569458, 1.132609486579895, 1.1331249475479126, 1.1422039270401, 1.1472387313842773, 1.155752182006836, 1.1727383136749268, 1.18197500705719, 1.

In [ ]:
result = pd.DataFrame()

In [ ]:
prediction = model2.predict(X_test)

In [ ]:
result['X_test'] = prediction
result['Y_test'] = y_test.tolist()

In [ ]:
result

In [ ]:
X_train.index = range(len(X_train))

In [ ]:
y_train.index = range(len(y_train))

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
# confusion_matrix(prediction,y_test)

# Test